In [2]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
# Afficher le répertoire courant
print("Répertoire courant :", os.getcwd())

Répertoire courant : C:\Users\flero\Downloads


In [8]:
# Changer de répertoire (remplacez le chemin par celui de votre dossier)
os.chdir(r'D:\HEC\Msc BI\Data Mining\Devoir Hiver')
print("Nouveau répertoire :", os.getcwd())

Nouveau répertoire : D:\HEC\Msc BI\Data Mining\Devoir Hiver


In [10]:

# Importation des données
# Ici, nous utilisons pd.read_csv() pour lire les fichiers texte.
# L'argument sep=r'\s+' précise que les colonnes sont séparées par des espaces.
mobiletrain = pd.read_csv("train.csv", sep=',')
mobiletest  = pd.read_csv("test.csv", sep=',')

# Affichage des dimensions du DataFrame mobiletrain
# L'attribut .shape renvoie un tuple (nombre de lignes, nombre de colonnes)
print("Dimensions de mobiletrain :", mobiletrain.shape)


Dimensions de mobiletrain : (5000, 17)


In [ ]:
mobiletrain.columns = mobiletrain.columns.str.strip()
mobiletest.columns = mobiletest.columns.str.strip()

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [14]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsOneClassifier


In [16]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score

# --- Nettoyage des colonnes ---
mobiletrain.columns = mobiletrain.columns.str.strip()
mobiletest.columns = mobiletest.columns.str.strip()

# --- Feature Engineering avec variables croisées et polynomiales ---
def add_features(df):
    df['avg_score_per_session'] = df['totscore'] / (df['numsessions'] + 1)
    df['avg_purchase_per_session'] = df['totpurchases'] / (df['numsessions'] + 1)
    df['playtime_per_day'] = df['totplaytime'] / (df['numdays'] + 1)
    df['score_per_day'] = df['totscore'] / (df['numdays'] + 1)
    df['purchase_efficiency'] = df['totpurchases'] / (df['totscore'] + 1)
    df['lives_per_day'] = df['numlives'] / (df['numdays'] + 1)
    df['elements_per_session'] = df['numelements'] / (df['numsessions'] + 1)
    df['difficulty_ratio'] = df['difflevel'] / (df['skill1'] + df['skill2'] + 1)
    df['acquisition_trend_interaction'] = df['acquis'] * df['trendpurchase']
    df['platform_trend_combo'] = df['opsys'] * df['trendsession']
    df['score_playtime_ratio'] = df['totscore'] / (df['totplaytime'] + 1)
    df['activity_ratio'] = df['numsessions'] / (df['numdays'] + 1)
    df['purchase_to_session_ratio'] = df['numpurchases'] / (df['numsessions'] + 1)

    # 👉 Variables polynomiales ajoutées (2e degré)
    df['score_x_sessions'] = df['totscore'] * df['numsessions']
    df['playtime_x_difficulty'] = df['totplaytime'] * df['difflevel']
    df['sessions_x_elements'] = df['numsessions'] * df['numelements']
    return df

mobiletrain = add_features(mobiletrain)
mobiletest = add_features(mobiletest)

# --- Encodage et préparation ---
X = pd.get_dummies(mobiletrain.drop(columns=['y']))
y = mobiletrain['y']
X_test = pd.get_dummies(mobiletest.drop(columns=['id']))
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# --- Encodage de la cible ---
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# --- Rééquilibrage automatique des classes ---
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
weights_dict = dict(enumerate(class_weights))

# --- LightGBM base model avec rééquilibrage ---
base_model = LGBMClassifier(
    objective='binary',
    n_estimators=628,
    learning_rate=0.104270746216993,
    max_depth=3, 
    min_child_samples=43,
    min_child_weight=10, 
    subsample=0.8370563423659869,
    colsample_bytree=0.8069913367080309,
    class_weight='balanced',
    random_state=42
)

# --- One-vs-One avec LightGBM ---
ovo_model = OneVsOneClassifier(base_model)

# --- Validation croisée locale (stratifiée) ---
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(ovo_model, X, y_encoded, cv=cv, scoring='accuracy', n_jobs=-1)

print(f"✅ Accuracy moyenne CV (5-fold): {cv_scores.mean():.5f}")
print(f"📊 Détail des scores par fold : {cv_scores}")

# --- Entraînement final sur 100% du dataset ---
ovo_model.fit(X, y_encoded)
# --- Prédiction finale sur X_test ---
y_test_pred_encoded = ovo_model.predict(X_test)
y_test_pred = label_encoder.inverse_transform(y_test_pred_encoded)

# --- Création du fichier de soumission ---
submission_ovo_lgbm = pd.DataFrame({
    'id': mobiletest['id'],
    'Prediction': y_test_pred
})
submission_ovo_lgbm.to_csv("submission_lightgbm_ovo_advanced_final.csv", index=False)
print("✅ Fichier 'submission_lightgbm_ovo_advanced_final.csv' généré avec succès ✅")


✅ Accuracy moyenne CV (5-fold): 0.80080
📊 Détail des scores par fold : [0.781 0.793 0.824 0.811 0.795]
[LightGBM] [Info] Number of positive: 1231, number of negative: 1240
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000490 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5206
[LightGBM] [Info] Number of data points in the train set: 2471, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further